## EJERCICIO 7 - PRACTICA 4

### Re-implementacion del ejercicio 4 (Balance.csv) utilizando un Multiperceptrón Tensorflow/Keras.

### Importo librerias keras

In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

import numpy as np
from sklearn import preprocessing, metrics, model_selection
import pandas as pd

In [ ]:
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
else:
    # configuración para notebook con instalación LOCAL
    FUENTES_DIR = '../Fuentes'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = '../Datos/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

Mounted at /content/drive/


### Importo dataset

In [ ]:
import pandas as pd      # para trabajar con archivos de datos csv, excel, etc: https://pandas.pydata.org/docs/getting_started/tutorials.html
import chardet           # para detectar la codificación de texto en archivos
import numpy as np


nombre_archivo = DATOS_DIR + 'Balance.csv'

#-- detectando la codificación de caracteres usada ----
with open(nombre_archivo, 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

# recupera el archivo en un objeto dataframe de pandas utilizando la codificación detectada
df = pd.read_csv(nombre_archivo, encoding=result['encoding'])
print(df)

    Balance  LeftWeight  LeftDist  RightWeight  RightDist
0         B           1         1            1          1
1         R           1         1            1          2
2         R           1         1            1          3
3         R           1         1            1          4
4         R           1         1            1          5
..      ...         ...       ...          ...        ...
620       L           5         5            5          1
621       L           5         5            5          2
622       L           5         5            5          3
623       L           5         5            5          4
624       B           5         5            5          5

[625 rows x 5 columns]


### Consigna

### Separo datos del dataset

In [ ]:
# %% separa atributos y clases
X = np.array(df.iloc[:,1:])  # recupera todas las columnas salvo la primera
Y = np.array(df.iloc[:,0])    # recupera solo la primer columna (valor a predecir)

# convertimos las etiquetas categóricas en numéricas
encoder = preprocessing.LabelEncoder()
Y = encoder.fit_transform(Y)
print(Y)

[0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 0 2 2 2 0 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 0 2 2 1 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 1 1 1 0 2 1 0 2 2 2 1 2 2 2 2 0 2 2 2 2 2 2 2 2 2 1 1 1 1 0 1 1 2 2 2 1
 2 2 2 2 1 2 2 2 2 0 2 2 2 2 1 0 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 1 1 1 0 2 1 0 2 2 2 1 2 2 2 2 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 0 2 2
 1 0 2 2 2 1 2 2 2 2 1 2 2 2 2 1 1 1 1 1 1 1 1 0 2 1 1 2 2 2 1 0 2 2 2 1 2
 2 2 2 1 1 1 1 1 1 1 1 1 0 1 1 1 2 2 1 1 2 2 2 1 0 2 2 2 1 1 0 2 2 1 2 2 2
 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 0 2 2 1 0 2 2 2 1 2 2 2 2 1
 2 2 2 2 1 1 1 1 1 1 1 1 1 2 1 1 0 2 2 1 1 2 2 2 1 2 2 2 2 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 2 1 1 0 2 2 1 1 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 2 2
 1 1 0 2 2 1 1 1 0 2 1 0 2 2 2 1 2 2 2 2 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1
 1 0 2 1 1 2 2 2 1 0 2 2 2 1 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 0 2 1 1 0 2
 2 1 1 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 2 1 1 1 2 2 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [ ]:
#--- CONJUNTOS DE ENTRENAMIENTO Y TESTEO ---
# utilizo el 70% de los datos para entrenar, y el otro 30% par testear
X_train, X_test, Y_train, Y_test = model_selection.train_test_split( \
        X, Y, test_size=0.30) #, random_state=42)

Y_trainB = to_categorical(Y_train)
print(Y_trainB[:5, :])

[[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [ ]:
normalizarEntrada = 1  # 1 si normaliza; 0 si no

if normalizarEntrada:
    # Escala los valores entre 0 y 1
    min_max_scaler = preprocessing.StandardScaler()
    X_train = min_max_scaler.fit_transform(X_train)
    X_test = min_max_scaler.transform(X_test)

Definicion del modelo

In [ ]:
# Crear un modelo de capas secuenciales
model=Sequential()

# Agregar las capas al modelo
# se agrega capa densa con 2 neuronas, con f. de activacion tanh:
# input_shape ---> es la cantidad de atributos de entrada existentes
model.add(Dense(2, input_shape=[4], activation='tanh'))
# se agrega capa densa con 3 neuronas, con f. de activacion sigmoid:
model.add(Dense(3, activation='sigmoid'))

# Imprimir un resumen del modelo
model.summary()



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 2)                   │              10 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19 (76.00 B)

 Trainable params: 19 (76.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Configuración para entrenamiento
#model.compile(optimizer=SGD(learning_rate=0.05), loss='mse', metrics='accuracy')
model.compile(optimizer=SGD(learning_rate=0.05), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train,Y_trainB, epochs=1000)
#epochs ---> "epocas": cantidad de veces que todos los datos de entrenamiento iteran 1 vez

Epoch 1/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1109 - loss: 1.2779   
Epoch 2/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4375 - loss: 1.0342 
Epoch 3/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6978 - loss: 0.8916  
Epoch 4/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8330 - loss: 0.7783  
Epoch 5/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8846 - loss: 0.6570  
Epoch 6/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8869 - loss: 0.6112  
Epoch 7/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8885 - loss: 0.5776  
Epoch 8/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8831 - loss: 0.5296  
Epoch 9/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8954 - loss: 0.4701  
Epoch 10/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9044 - loss: 0.4360  
Epoch 11/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8891 - loss: 0.4103
Epoch 12/1000
14/14 ━━━━━━━━━━━━

In [ ]:
# Calcular el error del modelo
score = model.evaluate(X_train, Y_trainB)
print('Error :', score[0])
print('Accuracy:', score[1])

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9375 - loss: 0.1566  
Error : 0.18423214554786682
Accuracy: 0.9221968054771423


In [ ]:
# predecir la salida del modelo
Y_pred = model.predict(X_train)
print(Y_pred[:5])

Y_pred2 = np.argmax(Y_pred,axis=1)
print(Y_pred2[:5])

print("%% aciertos X_train : %.3f" % metrics.accuracy_score(Y_train, Y_pred2))

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
[[5.4197121e-01 9.9962372e-01 1.6108539e-04]
 [6.1164087e-01 5.7998863e-03 9.9566334e-01]
 [6.9572002e-01 9.4572639e-01 1.3266464e-01]
 [5.0708497e-01 1.1632967e-01 7.5799382e-01]
 [4.9575144e-01 9.8199081e-01 5.5290889e-03]]
[1 2 1 2 1]
% aciertos X_train : 0.922


In [ ]:
report = metrics.classification_report(Y_train, Y_pred2)
print("Confusion matrix Training:\n%s" % report)

MM = metrics.confusion_matrix(Y_train, Y_pred2)
print("Confusion matrix:\n%s" % MM)

Confusion matrix Training:
              precision    recall  f1-score   support

           0       0.53      0.27      0.36        30
           1       0.93      0.96      0.94       197
           2       0.94      0.98      0.96       210

    accuracy                           0.92       437
   macro avg       0.80      0.74      0.75       437
weighted avg       0.91      0.92      0.91       437

Confusion matrix:
[[  8  11  11]
 [  7 189   1]
 [  0   4 206]]


# comparacion con ej4 (usando multiperceptron) --> no se bien como comparar, porque en ej 4 use maximo 2500 iteraciones, la amtriz de confusion en ej4 me quedo asi:

 Matriz de Confusión:
[[ 5  5  9]
 [ 1 81  0]
 [ 0  1 86]]

 se nota que con keras hubo mas ejemplos??? en los dos casos dividi 70%, 30%

       
          precision    recall  f1-score   support

    B       0.83      0.26      0.40        19
    L       0.93      0.99      0.96        82
    R       0.91      0.99      0.95        87


